In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DataReferenceConfiguration
from azureml.core.script_run_config import ScriptRunConfig
from azureml.widgets import RunDetails
import dotenv
%load_ext dotenv
%dotenv
import os

In [2]:
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
blob_datastore = ws.get_default_datastore()

In [4]:
blob_datastore.upload_files(['./dir_placeholder.txt'], target_path='bsdata', overwrite=True)

Uploading ./dir_placeholder.txt
Uploaded ./dir_placeholder.txt, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_47fc2227ece74874bc4cd7befbd7a748

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException

aml_compute_target = "cpucluster"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D3_V2",
                                                                min_nodes = 2, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Aml Compute attached")

found existing compute target.
Aml Compute attached


In [6]:
dataref_config = DataReferenceConfiguration(datastore_name=blob_datastore.name, 
                   path_on_datastore='bsdata', 
                   mode='mount', # download files from datastore to compute target
                   overwrite=True)

In [7]:
run_config = RunConfiguration(framework="PySpark")

# enable Docker 
run_config.environment.docker.enabled = True

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# auto-prepare the Docker image when used for execution (if it is not already prepared)
run_config.auto_prepare_environment = True

run_config.environment.docker.base_image = "microsoft/mmlspark:0.16" # "microsoft/mmlspark:0.12"

run_config.target = aml_compute
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas'])
run_config.node_count = 4

run_config.environment.spark.repositories = []
run_config.environment.spark.packages = []

run_config.data_references = {blob_datastore.name: dataref_config}

In [8]:
run_config.spark.configuration['spark.executor.memory'] = '10g'
#run_config.spark.configuration['spark.executor.instances'] = 3
run_config.spark.configuration['spark.driver.memory'] = '10g'

In [9]:
run_config.spark.configuration

{'spark.app.name': 'Azure ML Experiment',
 'spark.yarn.maxAppAttempts': 1,
 'spark.executor.memory': '10g',
 'spark.driver.memory': '10g'}

1_data_ingestion.py

In [ ]:
script_run_config = ScriptRunConfig(source_directory=".",
                                    script="1_data_ingestion.py",
                                    run_config=run_config,
                                    arguments=["--data_dir",str(blob_datastore.as_mount())])

run = Experiment(ws, 'script_run').submit(script_run_config)
run

In [ ]:
RunDetails(run).show()

2a_feature_engineering.py

In [ ]:
script_run_config = ScriptRunConfig(source_directory=".",
                                    script="2a_feature_engineering.py",
                                    run_config=run_config,
                                    arguments=["--data_dir",str(blob_datastore.as_mount())])

run = Experiment(ws, 'script_run').submit(script_run_config)
run

In [ ]:
RunDetails(run).show()

2b_model_building.py

In [12]:
script_run_config = ScriptRunConfig(source_directory=".",
                                    script="2b_model_building.py",
                                    run_config=run_config,
                                    arguments=["--data_dir",str(blob_datastore.as_mount())])

run = Experiment(ws, 'script_run').submit(script_run_config)
run

Experiment,Id,Type,Status,Details Page,Docs Page
script_run,script_run_1559867249_56f028b6,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [13]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

2a_feature_engineering.py for scoring

In [16]:
script_run_config = ScriptRunConfig(source_directory=".",
                                    script="2a_feature_engineering.py",
                                    run_config=run_config,
                                    arguments=["--data_dir",str(blob_datastore.as_mount()),
                                               "--features_table","scoring_data",
                                               "--start_date", "2015-11-15",
                                               "--to_date", "2016-04-30"])

run = Experiment(ws, 'script_run').submit(script_run_config)
run

Experiment,Id,Type,Status,Details Page,Docs Page
script_run,script_run_1559868277_f98e5a90,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [17]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

3b_model_scoring.py

In [24]:
script_run_config = ScriptRunConfig(source_directory=".",
                                    script="3b_model_scoring.py",
                                    run_config=run_config,
                                    arguments=["--data_dir",str(blob_datastore.as_mount())])

run = Experiment(ws, 'script_run').submit(script_run_config)
run

Experiment,Id,Type,Status,Details Page,Docs Page
script_run,script_run_1559870616_57fdd5cb,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [25]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…